In [1]:
# import cv2 to deal with resizing of images
import cv2 

# import numpy for working with arrays
import numpy as np

# import os for dealing with directories
import os

# import shuffle to mix up ordered data for better functionality
from random import shuffle

# import simple percentage bar for tasks
from tqdm import tqdm

# get data from filepaths
train_dir = "/Users/eoinmurnaghan/Downloads/all/train"
test_dir = "/Users/eoinmurnaghan/Downloads/all/test"

# make images 50x50 square
img_size = 50
lr = 1e-3

# 
model_name = "dogsvscats-{}-{}.model".format(lr, '2conv-basic')

print(model_name)

dogsvscats-0.001-2conv-basic.model


In [3]:
# creating classification function - [x,y] where x=catness, y=dogness
def label_img(img):
# eg. dog.93.png - [-3] denotes 3 letters in either cat or dog
    word_label = img.split('.')[-3]
    
#     [cat, no dog]
    if word_label == 'cat': return [1,0]
#     [no cat, dog]
    elif word_label == 'dog': return [0,1]

In [4]:
# make function to process train data
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img) 
        path = os.path.join(train_dir,img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size, img_size))
        training_data.append([np.array(img), np.array(label)])
    
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [5]:
# make function to process testing data
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(test_dir)):
        path = os.path.join(test_dir,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_))

In [22]:
train_data = create_train_data()

100%|██████████| 25000/25000 [01:18<00:00, 317.73it/s]


In [26]:
# NEW ADDED LINE
train_data = np.expand_dims(train_data, axis=0)

In [28]:
# define cnn for tensorflow
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


# 
convnet = input_data(shape=[None, img_size, img_size, 1], name='input')


# np.expand_dims(img, axis=0)

# image = array(img).reshape(1, 64,64,3)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [29]:
if os.path.exists('{}.meta'.format(model_name)):
    model.load(model_name)
    print('model loaded!')

In [30]:
# train using last 500 images
train = train_data[:-500]

# test on everything but last 500 images
test = train_data[-500:]

In [41]:
# feature sets, X
X = np.array([i[0] for i in train]).reshape(-1, img_size, img_size, 1)

# labels, y
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, img_size, img_size, 1)
test_y = [i[1] for i in test]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [20]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=model_name)

NameError: name 'test_x' is not defined